In [2]:
!pip install surya-ocr marker-pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.5 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [1]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded layout model datalab-to/surya_layout on device cuda with dtype torch.float16
Loaded texify model datalab-to/texify on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model datalab-to/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


In [2]:
import os
import hashlib


def calculate_hash(file_path):
    """Вычисляет SHA-256 хеш файла."""
    sha256 = hashlib.sha256()
    with open(file_path, 'rb') as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

def process_pdf_folder(folder_path, output_folder):
    """Обрабатывает папку с PDF файлами, конвертирует их в текст и исключает дубликаты."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    seen_hashes = set()
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            file_hash = calculate_hash(file_path)

            if file_hash not in seen_hashes:
                seen_hashes.add(file_hash)
                text = pdf_to_text(file_path)
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(text)
                print(f"Processed: {filename}")
            else:
                print(f"Duplicate found, skipping: {filename}")

def pdf_to_text(fname):
    rendered = converter(fname)
    text, _, images = text_from_rendered(rendered)
    return text

In [3]:
folder_path = 'tmp'
output_folder = 'tmp'
process_pdf_folder(folder_path, output_folder)

Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


Processed: Orlov_V_V_Osnovy_filosofii_Ch_1_Obschaya_filosofia_Vyp_1_Perm_2001.pdf
